<a href="https://colab.research.google.com/github/Leonnello/sjf-memory-placement-sim/blob/main/ITS150L_SA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ITS150L_FOPI01_1Q2425<br>
Members:<br>
Celina Therese Tabano<br>
Noel Allen Elises

In [ ]:
#### Imports
import random
import os
import signal
import pickle
import time

import ipywidgets as widgets


In [ ]:
class Process:
  position = None

  def __init__(self, pid, burst_time, memory_size):
    self.pid = pid
    self.burst_time = burst_time
    self.memory_size = memory_size

  # decrease remaining time
  def run(self):
    self.burst_time -= 1

    if self.burst_time == 0:
      self.position.isFree = True
      self.position = None


In [ ]:
class MemoryBlock:
  def __init__(self, start, end, isFree: bool = True):
    self.start = start
    self.end = end
    self.isFree = isFree

  def get_size(self):
    return self.end - self.start + 1

In [ ]:
##### GLOBAL VARIABLES
main_memory_size = 0

# counter for pid
pid_count = 1

# array of all process to be scheduled
current_processes = []

# array for all memory blocks
memory_blocks = []

run_time = 0
sc_time = 0
ch_time = 0

In [ ]:
def generate_blocks(size : int):
  # init rng
  random.seed()

  # select random num of blocks
  n_blocks = 0
  if size <= 3:
    n_blocks = 1
  elif size > 3 and size <= 500:
    n_blocks = random.randrange(3, 10)
  else:
    n_blocks = random.randrange(4, round(size/100))

  end_pos = random.sample(range(1,size), n_blocks)
  end_pos.sort()

  print(end_pos)

  # divide and create blocks
  for i in range(len(end_pos)):
    if i == 0:
      memory_blocks.append(MemoryBlock(0, end_pos[i]))
    elif i == len(end_pos) - 1:
      memory_blocks.append(MemoryBlock(memory_blocks[i-1].end + 1, size - 1))
    else:
      memory_blocks.append(MemoryBlock(memory_blocks[i-1].end + 1, end_pos[i]))


In [ ]:
# function to input new jobs to queue
bt_input = widgets.BoundedIntText(min=1, description=f"Burst Time")
size_input = widgets.BoundedIntText(min=1, description=f"Memory Size")
def new_job(e):
  global pid_count
  global bt_input, size_input

  burst_time = bt_input.value
  memory_size = size_input.value

  current_processes.append(Process(pid_count,burst_time, memory_size))
  print(f"Successfully added Process#{pid_count} {burst_time}ms {memory_size}kb")
  pid_count += 1

  # backup ig
  # while True:
  #   try:
  #     burst_time = int(input(f"Enter process#{pid_count} burst time: "))
  #     memory_size = int(input(f"Enter process#{pid_count} memory size: "))

  #     if burst_time == 0 or memory_size == 0:
  #       raise ValueError()

  #     current_processes.append(Process(pid_count,burst_time, memory_size))
  #     pid_count += 1
  #     break
  #   except ValueError:
  #     print("Invalid input. Enter non-zero whole number.")

In [ ]:
def pass_time(processes):
  global run_time, ch_time, sc_time

  # pause interval and increase run_time
  time.sleep(0.1)
  run_time += 1
  msg = ""

  if run_time != 0 and run_time % ch_time == 0:
    coalesce()
    repaint()
    display(gui)
    msg = f"CH at time: {run_time}"

  if run_time != 0 and run_time % sc_time == 0:
    compact()
    repaint()
    display(gui)
    msg += f"\nSC at time: {run_time}"

  if msg != "":
    print(msg)
    # pause so print won't zoom
    time.sleep(0.5)

  # pass time for each process that is already fitted in memory block
  for n in range(len(processes)):
    if processes[n] == None:
      continue

    if processes[n].position != None:
      processes[n].run()

    if processes[n].burst_time == 0:
      repaint()
      display(gui)
      print(f"Process#{processes[n].pid} completed")
      processes[n] = None




In [ ]:
# function to send current_processes for sjf scheduling
# run this whenever all the processes in a sjf batch is complete?
def sjf_scheduling(processes):
  global run_time, ch_time, sc_time
  # clear array for next batch
  global current_processes
  current_processes.clear()

  # sort processes based on Burst Time
  processes = sorted(processes, key=lambda process: process.burst_time)

  # fit jobs in memory blocks
  for i in range(len(processes)):
    if processes[i] == None:
      continue
    # pause interval and increase run_time
    time.sleep(0.3)
    print(f"PID {processes[i].pid}: {processes[i].burst_time}s {processes[i].memory_size}Kb")

    while not best_fit(processes[i]):
    # if can't fit, pass time till there is available hole
      pass_time(processes)
      # compact()

    # pass time every successful fit
    pass_time(processes)

  # pass time until all jobs are done
  for i in range(len(processes)):
    while processes[i] != None:
      pass_time(processes)

  print("Done executing")
  print(f"Time completed: {run_time}s")


In [ ]:
def best_fit(job_to_fit) -> bool:
  global memory_blocks
  difference = None
  block_index = -1

  for i in range(len(memory_blocks)):
    if memory_blocks[i].isFree and memory_blocks[i].get_size() >= job_to_fit.memory_size:
      current_diff = memory_blocks[i].get_size() - job_to_fit.memory_size
      if difference == None or current_diff < difference:
        difference = current_diff
        block_index = i

  # if fit is found
  if block_index != -1:
    # place job in the memory_block
    selected_block = memory_blocks[block_index]
    job_to_fit.position = selected_block
    selected_block.isFree = False

    end_pos = selected_block.end
    selected_block.end = selected_block.start + job_to_fit.memory_size - 1

    # create block for the remaining hole
    if difference != None and difference > 0:
      memory_blocks.append(MemoryBlock(selected_block.end + 1, end_pos))

    return True

  return False


In [ ]:
def is_in_array(array, element, *args) -> bool:
  for n in range(len(array)):
    match args:
      case ("start","end"):
        if array[n].start == element.start and array[n].end == element.end:
          return True

  return False

In [ ]:
# coalesce
def coalesce():
  global memory_blocks
  coalesced_blocks = []

  # sort by start #
  memory_blocks = sorted(memory_blocks, key=lambda block: block.start)

  if len(memory_blocks) == 1:
    coalesced_blocks = memory_blocks
  else:
    prev = None
    for i in range(len(memory_blocks) - 1):
      current = memory_blocks[i]
      next = memory_blocks[i+1]

      if prev is not None:
        # if previous was merged, update current
        if prev.isFree and prev.end == current.end:
          current = prev

      if current.isFree and next.isFree:
        # if prev is current, meaning prev and current will be merged
        # delete duplicate from final array
        if prev is current:
          coalesced_blocks.pop()

        current.end = next.end

      if not is_in_array(coalesced_blocks, current, "start", "end"):
        coalesced_blocks.append(current)

      if i == len(memory_blocks) - 2:
        if not is_in_array(coalesced_blocks, next, "start", "end"):
          coalesced_blocks.append(next)

      prev = current

  memory_blocks = coalesced_blocks.copy()


In [ ]:
def compact():
  global memory_blocks

  # #ignore function based on time interval
  # if run_time == 0 or run_time % 20 != 0: return

  memory_blocks = sorted(memory_blocks, key=lambda block: block.start)

  # count non-free blocks
  # process_count = sum(1 for block in memory_blocks if not block.isFree)

  #get all non-free blocks (biggest to smallest) then clear
  process_blocks = sorted([block for block in memory_blocks if not block.isFree], key=lambda block: block.get_size(), reverse = True)
  memory_blocks.clear()

  end = -1 # for first loop

  #change positions then append
  for block in process_blocks:
    start = end + 1
    end = block.get_size() - 1
    memory_blocks.append(MemoryBlock(start, end, False))

In [ ]:
isExit = False

def main():
  global current_processes, pid_count
  global memory_blocks, main_memory_size
  global run_time, sc_time, ch_time

  global isExit

  # pipe for interprocess comms
  pipe_r, pipe_w = os.pipe()

  # container to serialize/deserialize data for pipe
  data_container = {"processes" : current_processes}

  # while True:
  #   try:
  #     main_memory_size = int(input("Enter memory size of the program: "))
  #     if main_memory_size == 0:
  #       raise ValueError()
  #     break
  #   except ValueError:
  #     print("Invalid input. Enter non-zero whole number.")

  generate_blocks(main_memory_size)

  # while True:
  #   try:
  #     sc_time = int(input("Enter time for Compaction: "))
  #     ch_time = int(input("Enter time for Coalescing Hole: "))

  #     if sc_time == 0 or ch_time == 0:
  #       raise ValueError()
  #     break
  #   except ValueError:
  #     print("Invalid input. Enter non-zero whole number.")

  pid = os.fork()

  if pid == 0:
    # child process for the job simulation and execution

    # close write
    os.close(pipe_w)

    # child main process loop
    while True:
      read = os.read(pipe_r, 4096)
      if read:
        # deserialize data from pipe
        data = pickle.loads(read)

        # update current_processes in child process
        current_processes = data["processes"]

      # maybe start sjf_schedule when there's at least 5 process?
      # or just run it whenever it can ig
      # if len(current_processes) >= 3:
      #   sjf_scheduling(current_processes.copy())

      #   # display
      #   print(f"Time completed: {run_time}s")
      #   memory_blocks = sorted(memory_blocks, key=lambda block: block.start)
      #   for i in range(len(memory_blocks)):
      #     print(f"[{memory_blocks[i].start}, {memory_blocks[i].end}] {memory_blocks[i].isFree}")

    os.close(pipe_r)

  elif pid > 0:
    # parent process for inputting new jobs real time?

    # close read
    os.close(pipe_r)

    while True:
      # new_job()

      if len(current_processes) != 0:
        # serialize and pass updated data to child process
        serialized_data = pickle.dumps(data_container)
        os.write(pipe_w, serialized_data)

      ## :(
      # if isExit:
      #   # send exit signal to child then wait
      #   os.close(pipe_w)

      #   os.kill(pid, signal.SIGTERM)
      #   os.waitpid(pid, 0)
      #   print("Program exited successfully")
      #   break

      # user_prompt = input("Enter any to continue or 'exit' to quit: ")

      # if user_prompt.lower() == 'exit':
      if isExit:
        # send exit signal to child then wait
        os.close(pipe_w)

        os.kill(pid, signal.SIGTERM)
        os.waitpid(pid, 0)
        repaint()
        print("Program exited successfully")
        break



--------------------------------------------------------------------------------

In [ ]:
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, clear_output

In [ ]:
#part of static() - repaints static visuals
def repaintStatic(drawObj):
  # Set font (optional, use default if not available)
  try:
      font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 24)
  except IOError:
      font = ImageFont.load_default()

  # Draw the rectangle (outline only, no fill)
  drawObj.rectangle([(50, 50), (1050, 250)], outline="black", width=5)

  # Add text/label inside the rectangle
  text = "Memory"
  text_position = (50, 35)  # Position where the text will be placed

  # Add text to the image
  drawObj.text(text_position, text, fill="black", font=font)

In [ ]:
main_memory_size = 500

generate_blocks(main_memory_size)

# memory_blocks.clear()
# memory_blocks.append(MemoryBlock(0,99))
# memory_blocks.append(MemoryBlock(100,149))
# memory_blocks.append(MemoryBlock(150,199))
# memory_blocks.append(MemoryBlock(200,249, False))
# memory_blocks.append(MemoryBlock(250,349))
# memory_blocks.append(MemoryBlock(350,499, False))


[95, 157, 207, 318, 368, 376]


In [ ]:
#v2 using html - implementation below

import ipywidgets as widgets
from IPython.display import display, HTML

#displays memory in a single frame
def repaint():
  global memory_blocks
  global main_memory_size
  currentPos = 0

  width_ratio = main_memory_size/ 1000
  blocks_html = ''

  for block in memory_blocks:
    p2 = int(block.get_size() * width_ratio)
    fill_color = 'transparent' if block.isFree else '#FFAE00'

    # label_pos = (currentPos[0] + 5, currentPos[1] + 90)
    # get_status = "Free" if block.isFree else "Allocated"
    # drawObj.text(label_pos, f"[{block.start},{block.end}]\n{get_status}", fill="black")

    blocks_html += f'<div style="border: 5px solid #000000; width: {p2}px; height: 100px; background-color: {fill_color}; position: absolute; top: 48px; left: {currentPos}px;"></div>'

    #increment pointer with block size
    currentPos += int(block.get_size() * width_ratio)

    # with visual:
    #visual.clear_output(wait=True)
    clear_output(wait=True)

  # statics
  text_memory = '<h2>Memory</h2>'
  memory_rect = f'<div style="border: 5px solid #000000; width: 1000px; height: 100px;">{blocks_html}</div>'

  html_content = widgets.HTML(text_memory + memory_rect)
  display(html_content)
  # print("repaint complete.")

In [ ]:
## Exit using button
# di siya nagwork :(

exit_button = widgets.Button(description="Press to Exit")
output = widgets.Output()
isExit = False

def on_exit(e):
  global isExit
  with output:
    print("Exiting...")
  isExit = True
  exit_button.disabled = True
  # clear_ouptut(wait=True)

def on_run(e):
  if len(current_processes) != 0:
    sjf_scheduling(current_processes.copy())

exit_button.on_click(on_exit)

add_job_button = widgets.Button(description="Add Job")
add_job_button.on_click(new_job)
run_button = widgets.Button(description="Run SJF")
run_button.on_click(on_run)

In [ ]:
repaint()
## this is done i think, fingers crossed
"""plan
-display "html gui + input"
-take user input
-while entering dif processes, user can input/press 'start', exit button appears as well
-clear_output then display simulation
-repaint only gets called when change in memory block happens (in coalesce, compact, and when burst hits zero)
-after sim, display "html gui + input"
"""
mem_input = widgets.BoundedIntText(value=1000, min=1, max=1000, description="Memory Size(kb):")
compact_input = widgets.BoundedIntText(value=20, min=1, description="Compaction Interval(ms):")
coalesce_input = widgets.BoundedIntText(value=5, min=1, description="Coalesce Interval(ms):")

main_memory_size = mem_input.value
sc_time = compact_input.value
ch_time = coalesce_input.value

#enter burst
#enter size mb

display(widgets.HBox([mem_input, compact_input, coalesce_input]))

gui = widgets.Output()
with gui:
  display(widgets.HBox([bt_input, size_input, add_job_button]))
  display(widgets.HBox([run_button, exit_button]))

display(gui)

HTML(value='<h2>Memory</h2><div style="border: 5px solid #000000; width: 1000px; height: 100px;"><div style="b…

Output()

Process#3 completed
Done executing
Time completed: 22s


In [ ]:
## ISSUE:
# nagcoconflict yung clear_output sa prompt. Di ma-exit yung program.
# I tried using widget.button and widget.text sa exit kaso siya nareread :(
# tapos yung sa output nag-ooverlap yung old instances :(

# display(exit_button, output)

#### So bali mawawala na main()?
# main()


HTML(value='<h2>Memory</h2><div style="border: 5px solid #000000; width: 1000px; height: 100px;"><div style="b…

Output()

Done executing
Time completed: 6s


------------------------------disregard test area below-------------------------

In [ ]:
#deprecated
visual = widgets.Output(layout={'border': '1px solid black'})
image = Image.new("RGB", (1050, 300), color="white")
drawObj = ImageDraw.Draw(image)
display(visual)

def repaint2():
  global memory_blocks
  global main_memory_size
  currentPos = (50, 50)

  width_ratio = 1000/ main_memory_size

  repaintStatic(drawObj)

  for block in memory_blocks:

    p2 = (currentPos[0] + int(block.get_size() * width_ratio), 250)


    fill_color = None if block.isFree else 'blue'
    drawObj.rectangle([currentPos, p2], outline='black', fill=fill_color, width=4)

    label_pos = (currentPos[0] + 5, currentPos[1] + 90)
    # get_status = "Free" if block.isFree else "Allocated"
    # drawObj.text(label_pos, f"[{block.start},{block.end}]\n{get_status}", fill="black")

    #increment pointer with block size
    currentPos = (currentPos[0] + int(block.get_size() * width_ratio), currentPos[1])
    # currentPos = (currentPos[0] + block.get_size(), currentPos[1])

    # with visual:
    #visual.clear_output(wait=True)
    clear_output(wait=True)
    display(image)

    #frame rate
    time.sleep(1)
  print("repaint complete.")



In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import random

# Function to change the background color of the rectangle
def change_color():
    color = random.choice(["#33FF57", "#FF5733", "#3357FF", "#FFD700", "#FF1493"])  # Random color
    rectangle.value = f'<div style="background-color: {color}; width: 200px; height: 100px;"></div>'

# Create the initial rectangle
rectangle = widgets.HTML(value='<div style="background-color: #33FF57; width: 200px; height: 100px;"></div>')

# Create a button to trigger color change
button = widgets.Button(description="Change Color")

# Link the button to the color change function
button.on_click(change_color)

# Display the widget and button
display(rectangle, button)

for i in range(10):
  change_color()
  time.sleep(1)
  print("change")

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Create an HTML widget with a parent rectangle containing multiple child rectangles
html_widget = widgets.HTML(value="""
<div style="width: 500px; height: 300px; border: 2px solid black; position: relative; ">
    <!-- First Child Rectangle -->
    <div style="width: 100px; height: 50px; background-color: #33FF57; position: absolute; top: 0px; left: 0px;"></div>

    <!-- Second Child Rectangle -->
    <div style="width: 120px; height: 60px; background-color: #FF5733; position: absolute; top: 100px; left: 50px;"></div>

    <!-- Third Child Rectangle -->
    <div style="width: 150px; height: 80px; background-color: #3357FF; position: absolute; top: 180px; left: 120px;"></div>
</div>
""")

# Display the HTML widget
display(html_widget)
time.sleep(5)
display(exit_button)


In [ ]:
html_test = widgets.HTML(value=
                         """
  <!DOCTYPE html>
  <html lang="en">
  <head>
      <meta charset="UTF-8">
      <meta name="viewport" content="width=device-width, initial-scale=1.0">
      <title>Color Change Widget</title>
      <style>
          #colorRectangle {
              width: 200px;
              height: 100px;
              border: 2px solid #000;
              margin-top: 20px;
          }
      </style>
  </head>
  <body>

  <h2>Enter 5 Colors</h2>
  <form id="colorForm">
      <label for="color1">Color 1: </label>
      <input type="text" id="color1" placeholder="e.g., #FF5733"><br><br>
      <label for="color2">Color 2: </label>
      <input type="text" id="color2" placeholder="e.g., rgb(0, 255, 0)"><br><br>
      <label for="color3">Color 3: </label>
      <input type="text" id="color3" placeholder="e.g., blue"><br><br>
      <label for="color4">Color 4: </label>
      <input type="text" id="color4" placeholder="e.g., #3498db"><br><br>
      <label for="color5">Color 5: </label>
      <input type="text" id="color5" placeholder="e.g., rgba(255, 0, 0, 0.5)"><br><br>

      <button type="button" onclick="changeColor()">Change Color</button>
  </form>

  <div id="colorRectangle"></div>

  <script>
      function changeColor() {
          // Get the color values from the text input fields
          const color1 = document.getElementById('color1').value;
          const color2 = document.getElementById('color2').value;
          const color3 = document.getElementById('color3').value;
          const color4 = document.getElementById('color4').value;
          const color5 = document.getElementById('color5').value;

          // Collect all colors in an array
          const colors = [color1, color2, color3, color4, color5];

          // Filter out empty inputs (in case user doesn't fill some fields)
          const validColors = colors.filter(color => color.trim() !== "");

          // If there are no valid colors, alert the user and return
          if (validColors.length === 0) {
              alert("Please enter at least one color.");
              return;
          }

          // Select a random color from the valid colors
          const randomColor = validColors[Math.floor(Math.random() * validColors.length)];

          // Change the background color after 1 second delay
          setTimeout(function() {
              document.getElementById('colorRectangle').style.backgroundColor = randomColor;
          }, 1000);  // Delay of 1 second (1000 milliseconds)
      }
  </script>

  </body>
  </html>

  """
)

display(html_test)

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import random

# Memory simulation: for simplicity, we'll use a fixed memory size and allocate memory to processes
MEMORY_SIZE = 1000  # 1000 units of memory
memory_block = [0] * MEMORY_SIZE  # A simple memory block (0 represents free space)

# Function to simulate memory allocation
def allocate_memory(process_id, memory_required):
    """Simulate memory allocation for the process."""
    for i in range(MEMORY_SIZE - memory_required + 1):  # +1 to ensure we don't go out of range
        if all(x == 0 for x in memory_block[i:i + memory_required]):  # Check for a free block
            memory_block[i:i + memory_required] = [process_id] * memory_required  # Allocate memory
            return True
    return False  # If no memory found

# SJF scheduling algorithm
def sjf_scheduler(processes):
    """Simulate the Shortest Job First (SJF) scheduling."""
    processes.sort(key=lambda x: x[1])  # Sort by burst time (shortest first)

    result = []
    memory_allocations = []

    for pid, burst_time, mem_req in processes:
        if allocate_memory(pid, mem_req):
            memory_allocations.append(f"Process {pid} allocated {mem_req} units of memory.")
        else:
            memory_allocations.append(f"Process {pid} could not be allocated memory.")
        result.append(f"Process {pid} completed after {burst_time} units.")

    return result, memory_allocations

# Widgets for process input
num_processes = widgets.IntSlider(value=3, min=1, max=10, step=1, description='Processes:')
display(num_processes)

# Process ID, Burst Time, and Memory requirements input
process_inputs = []
process_widgets = []

def update_inputs(change):
    """Update the process input fields when number of processes changes."""
    # Clear previous widgets
    for widget in process_widgets:
        widget.layout.display = 'none'

    process_widgets.clear()
    process_inputs.clear()

    for i in range(num_processes.value):
        pid_input = widgets.IntText(value=i+1, description=f'PID {i+1}:', disabled=False)
        burst_time_input = widgets.IntText(value=random.randint(1, 10), description='Burst Time:', disabled=False)
        memory_input = widgets.IntText(value=random.randint(50, 100), description='Memory Req:', disabled=False)

        # Add widgets to the list
        process_inputs.append((pid_input, burst_time_input, memory_input))

        # Display them
        widget_group = widgets.VBox([pid_input, burst_time_input, memory_input])
        process_widgets.append(widget_group)
        display(widget_group)

# Observe the number of processes change to update inputs dynamically
num_processes.observe(update_inputs, names='value')

# Submit button to run scheduler
def on_submit_button_click(b):
    """Run SJF Scheduler and show results."""
    processes = []

    for pid_input, burst_time_input, memory_input in process_inputs:
        processes.append((pid_input.value, burst_time_input.value, memory_input.value))

    # Clear memory block before running the scheduler
    global memory_block
    memory_block = [0] * MEMORY_SIZE  # Reset memory

    scheduling_result, memory_result = sjf_scheduler(processes)

    # Display results
    scheduling_output = "\n".join(scheduling_result)
    memory_output = "\n".join(memory_result)

    display(HTML(f"<h3>SJF Scheduling Results:</h3><pre>{scheduling_output}</pre>"))
    display(HTML(f"<h3>Memory Allocation Results:</h3><pre>{memory_output}</pre>"))

submit_button = widgets.Button(description="Run SJF Scheduler")
submit_button.on_click(on_submit_button_click)
display(submit_button)

